In [247]:
import pandas as pd
import numpy as np


data = pd.read_csv(r"C:\Users\tomas\Desktop\Proyectos\Titanic\train.csv")
test = pd.read_csv(r"C:\Users\tomas\Desktop\Proyectos\Titanic\test.csv") 
Ejemplo = pd.read_csv(r"C:\Users\tomas\Desktop\Proyectos\Titanic\gender_submission.csv")

In [248]:
print(data.isnull().sum())
print(data.shape)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
(891, 12)


In [249]:
# Reemplazo las edades faltantes con la mediana de las mismas

data["Age"] = data["Age"].fillna(data["Age"].median())

# Reemplazo cabin con 1 si hay datos, 0 si no.

data["Cabin"] = np.where(data["Cabin"].isnull(), 0, 1)

# Los casos de Embarked por su clase mayoritaria

print(data["Embarked"].value_counts())

data["Embarked"] = data["Embarked"].fillna("S")

S    644
C    168
Q     77
Name: Embarked, dtype: int64


In [250]:
# Preparo los datos como x e y

y = data["Survived"]
X = data.drop(["Survived","PassengerId","Name","Ticket"],axis=1)

# Hago booleano las variables categóricas

X=pd.get_dummies(X)





In [251]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=3041997)

# Los modelos suelen correr mejor cuando se los escala, así que hacemos eso
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [252]:
# Vamos a correr unos modelitos para analizarlos, aunque seguramente nos quedemos con random forest 
#porque suele ser el que mejor dá.

# Vamos a usar la curva roc para medir que modelo es mejor (hay un paper masomenos conocido que aclara que es mejor
#que accuracy. Lo dejo en la bibliografía).

from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score

#Naive Bayes

from sklearn.naive_bayes import GaussianNB
NB = GaussianNB()
print("Modelo en train con cross validation tiene una media de {}".format(cross_val_score(NB, X_train, y_train, scoring="roc_auc", cv = 3).mean()))
print("Modelo en train con cross validation tiene una desvio de {}".format(cross_val_score(NB, X_train, y_train, scoring="roc_auc", cv = 3).std()))


NB.fit(X_train,y_train)

y_pred=NB.predict(X_test)

print("Resultado en test {}".format(roc_auc_score(y_test,y_pred)))




Modelo en train con cross validation tiene una media de 0.8263119859268767
Modelo en train con cross validation tiene una desvio de 0.008454556405291101
Resultado en test 0.7869565217391306


In [253]:
# Regresión logística

from sklearn.linear_model import LogisticRegression
rl= LogisticRegression(max_iter=10000000000000)
print("Modelo en train con cross validation tiene una media de {}".format(cross_val_score(rl, X_train, y_train, scoring="roc_auc", cv = 3).mean()))
print("Modelo en train con cross validation tiene una desvio de {}".format(cross_val_score(rl, X_train, y_train, scoring="roc_auc", cv = 3).std()))


rl.fit(X_train,y_train)

y_pred=rl.predict(X_test)

print("Resultado en test {}".format(roc_auc_score(y_test,y_pred)))

# Un poquito mejor




Modelo en train con cross validation tiene una media de 0.837808206152237
Modelo en train con cross validation tiene una desvio de 0.005587220546777577
Resultado en test 0.7945652173913043


In [254]:
# Vecinos mas cercanos. A parir de acá ya vamos a optimizar los hiper parámetros

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

par = {"n_neighbors":[3,5,7,9,11], "algorithm" :["auto", "ball_tree", "kd_tree", "brute"]}

KNN= KNeighborsClassifier()

clf = GridSearchCV(KNN, par,scoring="roc_auc")

clf.fit(X_train,y_train)

KNN = KNeighborsClassifier(**clf.best_params_)


print("Modelo en train con cross validation tiene una media de {}".format(cross_val_score(KNN, X_train, y_train, scoring="roc_auc", cv = 3).mean()))
print("Modelo en train con cross validation tiene una desvio de {}".format(cross_val_score(KNN, X_train, y_train, scoring="roc_auc", cv = 3).std()))


KNN.fit(X_train,y_train)

y_pred=KNN.predict(X_test)

print("Resultado en test {}".format(roc_auc_score(y_test,y_pred)))


# Todavía mejor

Modelo en train con cross validation tiene una media de 0.839927494888984
Modelo en train con cross validation tiene una desvio de 0.02148037385206852
Resultado en test 0.7997584541062802


In [255]:
# Maquinas de soporte vectorial

from sklearn.svm import SVC

par = {"C":[0.001,0.01,0.1,1,10], "kernel" :["linear", "poly", "rbf", "sigmoid"],
      "gamma":["scale", "auto"]}

svc= SVC()

clf = GridSearchCV(svc, par,scoring="roc_auc")

clf.fit(X_train,y_train)

svc = SVC(**clf.best_params_)


print("Modelo en train con cross validation tiene una media de {}".format(cross_val_score(svc, X_train, y_train, scoring="roc_auc", cv = 3).mean()))
print("Modelo en train con cross validation tiene una desvio de {}".format(cross_val_score(svc, X_train, y_train, scoring="roc_auc", cv = 3).std()))


svc.fit(X_train,y_train)

y_pred=svc.predict(X_test)

print("Resultado en test {}".format(roc_auc_score(y_test,y_pred)))

# Obviamente mejor

Modelo en train con cross validation tiene una media de 0.8433024295155233
Modelo en train con cross validation tiene una desvio de 0.00906730908966201
Resultado en test 0.8254830917874397


In [256]:
# Arbol de decisión (uno solito)

from sklearn.tree import DecisionTreeClassifier

par = {"criterion":["gini", "entropy"], "splitter" :["best","random"],
      "max_depth":list(range(1,12)),
      "max_features" :["auto", "sqrt", "log2"]}

DTC= DecisionTreeClassifier()

clf = GridSearchCV(DTC, par,scoring="roc_auc")

clf.fit(X_train,y_train)

DTC = DecisionTreeClassifier(**clf.best_params_)


print("Modelo en train con cross validation tiene una media de {}".format(cross_val_score(DTC, X_train, y_train, scoring="roc_auc", cv = 3).mean()))
print("Modelo en train con cross validation tiene una desvio de {}".format(cross_val_score(DTC, X_train, y_train, scoring="roc_auc", cv = 3).std()))


DTC.fit(X_train,y_train)

y_pred=DTC.predict(X_test)

print("Resultado en test {}".format(roc_auc_score(y_test,y_pred)))

# Tiene sentido que dé peor. Las implementaciones como random forest suelen dar mejor.

Modelo en train con cross validation tiene una media de 0.7837298530880045
Modelo en train con cross validation tiene una desvio de 0.018629043682777995
Resultado en test 0.7472222222222222


In [257]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier

par = {'n_estimators': [100, 700,100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8,9,10],
    'criterion' :['gini', 'entropy']}

RF= RandomForestClassifier()

clf = GridSearchCV(RF, par,scoring="roc_auc")

clf.fit(X_train,y_train)

RF = RandomForestClassifier(**clf.best_params_)


print("Modelo en train con cross validation tiene una media de {}".format(cross_val_score(RF, X_train, y_train, scoring="roc_auc", cv = 3).mean()))
print("Modelo en train con cross validation tiene una desvio de {}".format(cross_val_score(RF, X_train, y_train, scoring="roc_auc", cv = 3).std()))


RF.fit(X_train,y_train)

y_pred=RF.predict(X_test)

print("Resultado en test {}".format(roc_auc_score(y_test,y_pred)))


# El que mejor dió hasta ahora

Modelo en train con cross validation tiene una media de 0.8492915894071221
Modelo en train con cross validation tiene una desvio de 0.01917193075033618
Resultado en test 0.8235507246376812


In [258]:
# Hagamos Boosting ahorita con árboles

from sklearn.ensemble import AdaBoostClassifier

Arbol1 = DecisionTreeClassifier(max_depth = 3,max_features="sqrt")
Arbol2 = DecisionTreeClassifier(max_depth = 3,max_features="log2")
Arbol3 = DecisionTreeClassifier(max_depth = 4,max_features="sqrt")
Arbol4 = DecisionTreeClassifier(max_depth = 4,max_features="log2")
Arbol5 = DecisionTreeClassifier(max_depth = 5,max_features="sqrt")
Arbol6 = DecisionTreeClassifier(max_depth = 5,max_features="log2")
Arbol7 = DecisionTreeClassifier(max_depth = 6,max_features="sqrt")
Arbol8 = DecisionTreeClassifier(max_depth = 6,max_features="log2")
Arbol9 = DecisionTreeClassifier(max_depth = 7,max_features="sqrt")
Arbol10 = DecisionTreeClassifier(max_depth = 7,max_features="log2")
Arbol11 = DecisionTreeClassifier(max_depth = 8,max_features="sqrt")
Arbol12 = DecisionTreeClassifier(max_depth = 8,max_features="log2")
Arbol13 = DecisionTreeClassifier(max_depth = 9,max_features="sqrt")
Arbol14 = DecisionTreeClassifier(max_depth = 9,max_features="log2")
Arbol15 = DecisionTreeClassifier(max_depth = 10,max_features="sqrt")
Arbol16 = DecisionTreeClassifier(max_depth = 10,max_features="log2")
par = {'base_estimator': [Arbol1,Arbol2,Arbol3,Arbol4,Arbol5,Arbol6,Arbol7,Arbol8,Arbol9,Arbol10,
                         Arbol11,Arbol12,Arbol13,Arbol14,Arbol15,Arbol16],
    'n_estimators': [50,60,70,80,90,100],
    'learning_rate' : [0.01,0.1,1,10]}

ADA= AdaBoostClassifier()

clf = GridSearchCV(ADA, par,scoring="roc_auc")

clf.fit(X_train,y_train)

ADA = AdaBoostClassifier(**clf.best_params_)


print("Modelo en train con cross validation tiene una media de {}".format(cross_val_score(ADA, X_train, y_train, scoring="roc_auc", cv = 3).mean()))
print("Modelo en train con cross validation tiene una desvio de {}".format(cross_val_score(ADA, X_train, y_train, scoring="roc_auc", cv = 3).std()))


ADA.fit(X_train,y_train)

y_pred=ADA.predict(X_test)

print("Resultado en test {}".format(roc_auc_score(y_test,y_pred)))

# Por lo pronto es el 2do modelo que mejor dió.

Modelo en train con cross validation tiene una media de 0.8211581800028526
Modelo en train con cross validation tiene una desvio de 0.016234996566470723
Resultado en test 0.8298309178743962


In [259]:
# Bueno, ya probé bastante. Vamos a predecir

print(test.isnull().sum())

Id = test["PassengerId"]
# Reemplazo las edades faltantes con la mediana de las mismas

test["Age"] = test["Age"].fillna(test["Age"].median())

# Reemplazo cabin con 1 si hay datos, 0 si no.

test["Cabin"] = np.where(test["Cabin"].isnull(), 0, 1)

# Lo mismo que con edades en Fare

test["Fare"] = test["Fare"].fillna(test["Fare"].median())



X = test.drop(["PassengerId","Name","Ticket"],axis=1)

# Hago booleano las variables categóricas

X=pd.get_dummies(X)

X = scaler.transform(X)








PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


RandomForestClassifier(max_depth=4, max_features='log2')

In [260]:
RF.fit(X_train,y_train)
y_pred = RF.predict(X)



In [261]:
Ejemplo

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [262]:
d = {'PassengerId': Id, 'Survived': y_pred}
df = pd.DataFrame(data=d)
df.to_csv("Prueba.csv",index=False)

#Y esto obtuvo 79% en kaggle